In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from keras.utils.data_utils import get_file
from os.path import join
import json
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:

def remove_top_layer(model):
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    
def feed_forward(model, img):
    img = img_to_array(img)
    img= np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return model.predict(img)

def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None, figtitle=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    if figtitle:
        f.suptitle(figtitle)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

def get_prediction_label(predictions, num_preds):
    classes = get_classes()
    
    result = list(zip(predictions[0], classes))
    result = sorted(result, key= lambda elem: elem[0], reverse=True)
    
    return result[:num_preds]
    
    

In [3]:



class AlikeFinder():
    def __init__(self):
        self.prepare_model()
        
    def prepare_model(self):
        model = InceptionV3(weights='imagenet')
        remove_top_layer(model)
        self.model = model
        
    def get_features_for_path(self, img_path):
        image = load_img(img_path)
        return feed_forward(self.model, image)
    
    def get_features_for_image(self, image):
        return feed_forward(self.model, image)
        
original_model = InceptionV3(weights='imagenet')
alike_finder = AlikeFinder()        

In [6]:
image_url = 'https://www2.deloitte.com/content/dam/Deloitte/uk/Images/promo_images/Campaign/sport/uk-deloitte-sbg-football-us-fans-promo.jpg'
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))


orig_result = feed_forward(original_model, img)
original_predictions = decode_predictions(orig_result)

plots([img], titles = [str(elem[1] + " " + str(elem[2]) for elem in original_predictions[0])])



SyntaxError: unexpected EOF while parsing (<ipython-input-6-2cb28972ea93>, line 9)